In [7]:
from typing import List
import os
import re
import time

import numpy as np
import pandas as pd
from pandas import DataFrame
import requests

In [8]:
import os

from util.widgets import generate_ym, generate_ymd, generate_ymd_HMS, generate_i

ym = generate_ym()
g = 'cargohose_herren'
bd = f'{g}/{ym}'
bd_a = f'{bd}/a'
b_d = f'{g}_{ym}'

os.makedirs(bd, exist_ok=True)
os.makedirs(bd_a, exist_ok=True)

ymd = generate_ymd()
ymd_HMS = generate_ymd_HMS()
i = '(08-09)'
bd_a_img = f'{bd_a}/{ymd}'
os.makedirs(bd_a_img, exist_ok=True)

In [9]:
fp_erp = f'{bd_a}/product_20240729110524847.xls'
erp_df: DataFrame = pd.read_excel(fp_erp)

erp_df

,SKU,父SKU,产品缩略图,产品类型,标题,产品名称,描述,颜色,尺寸,亚马逊尺寸,...,S3图片链接 8缩略图,S3图片链接 8,S3图片链接 9缩略图,S3图片链接 9,子图链接,长,宽,高,尺码表,印花名称
0,AHY240625001,NaN,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Male Summer Cargo Pants Solid Color Pocket Tet...,男士多口袋纽扣五分工装短裤,"<h1 align=""center""><span style=""font-size:14pt...","Black,Yellow,Green,Coffee,Khaki,Grey","S,M,L,XL,XXL,XXXL",NaN,...,NaN,https://oz1xbcxa3ey1u5sz.imgsapp.com/afsd/1935...,NaN,https://o09tclpn5hvfql24.imgsapp.com/afsd/1935...,NaN,30,20,3,Size: S Waist: 74cm/29.13'' Hip: 121cm/47.64''...,其他
1,AHY240625001BKL,AHY240625001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Male Summer Cargo Pants Solid Color Pocket Tet...,男士多口袋纽扣五分工装短裤 黑色 L-31,"<h1 align=""center""><span style=""font-size:14pt...",Black,L-31,NaN,...,NaN,NaN,NaN,NaN,NaN,33,24,1,Size: S Waist: 74cm/29.13'' Hip: 121cm/47.64''...,NaN
2,AHY240625001BKL2,AHY240625001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Male Summer Cargo Pants Solid Color Pocket Tet...,男士多口袋纽扣五分工装短裤 黑色XXL-33,"<h1 align=""center""><span style=""font-size:14pt...",Black,XXL-33,NaN,...,NaN,NaN,NaN,NaN,NaN,33,24,1,Size: S Waist: 74cm/29.13'' Hip: 121cm/47.64''...,NaN
3,AHY240625001BKL3,AHY240625001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Male Summer Cargo Pants Solid Color Pocket Tet...,男士多口袋纽扣五分工装短裤 黑色 XXXL-34,"<h1 align=""center""><span style=""font-size:14pt...",Black,XXXL-34,NaN,...,NaN,NaN,NaN,NaN,NaN,33,24,1,Size: S Waist: 74cm/29.13'' Hip: 121cm/47.64''...,NaN
4,AHY240625001BKM,AHY240625001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Male Summer Cargo Pants Solid Color Pocket Tet...,男士多口袋纽扣五分工装短裤 黑色 M-30,"<h1 align=""center""><span style=""font-size:14pt...",Black,M-30,NaN,...,NaN,NaN,NaN,NaN,NaN,33,24,1,Size: S Waist: 74cm/29.13'' Hip: 121cm/47.64''...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,ZZS210511744GYL5,ZZS210511744,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Casual Straight Summer Multi Pocket Men's Pant...,男士夏季多口袋直筒休闲裤工装裤长裤 灰色 XXXXXL,"<h1 align=""center""><span style=""font-size:14pt...",Grey,XXXXXL,5X-Large,...,NaN,https://beaqdilces304it4.imgsapp.com/afsd/8802...,NaN,https://xmvlodrvwlswrqur.imgsapp.com/afsd/8802...,NaN,30,23,1,Size: S US: 4 UK: 8 EU: 34 Waist: 77cm/30.31''...,NaN
684,ZZS210511744GYS,ZZS210511744,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Casual Straight Summer Multi Pocket Men's Pant...,男士夏季多口袋直筒休闲裤工装裤长裤 灰色 S,"<h1 align=""center""><span style=""font-size:14pt...",Grey,S,Small,...,NaN,https://uw04vmkwhu29sfli.imgsapp.com/afsd/8802...,NaN,https://yehpc88ffcmmurw8.imgsapp.com/afsd/8802...,NaN,30,23,1,Size: S US: 4 UK: 8 EU: 34 Waist: 77cm/30.31''...,NaN
685,ZZS210511744KHL4,ZZS210511744,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Casual Straight Summer Multi Pocket Men's Pant...,男士夏季多口袋直筒休闲裤工装裤长裤 卡其色 XXXXL,"<h1 align=""center""><span style=""font-size:14pt...",Khaki,XXXXL,4X-Large,...,NaN,https://oz7ag1f8cmj8r9t2.imgsapp.com/afsd/8802...,NaN,https://4g2e6dzvc73tya1m.imgsapp.com/afsd/8802...,NaN,30,23,1,Size: S US: 4 UK: 8 EU: 34 Waist: 77cm/30.31''...,NaN
686,ZZS210511744KHL5,ZZS210511744,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Casual Straight Summer Multi Pocket Men's Pant...,男士夏季多口袋直筒休闲裤工装裤长裤 卡其色 XXXXXL,"<h1 align=""center""><span style=""font-size:14pt...",Khaki,XXXXXL,5X-Large,...,NaN,https://pxqulwuv6hvf1d9p.imgsapp.com/afsd/8802...,NaN,https://idkvyw32unyuuhqc.imgsapp.com/afsd/8802...,NaN,30,23,1,Size: S US: 4 UK: 8 EU: 34 Waist: 77cm/30.31''...,NaN


In [10]:
def generate_amazon_sku(s: str, c: str, n: str, b: int) -> str:
    """

    :param s: ERP的sku
    :param c: 仓库编号
    :param n: 节点
    :param b: 父sku批号
    :return:
    """
    p = re.compile(r'^(?P<p>[A-Z]+)(?P<d>\d{6})(?P<n>\d+)(?P<cs>.*?)$', re.S)
    # for i in p.match(s).groups():
    #     print(i)
    t = ymd_HMS[2:].replace(' ', '-').replace(':', '-')
    cs = p.match(s).group('cs')
    result = f'{c}-{n}-{t}-{b:03d}-{cs}'.strip('-')
    return result


s1 = 'BYJ240716009BWL2'
# MKT-DE-24-07-18-001-BWL2
generate_amazon_sku(s1, 'FRL', 'DE', 1)

'FRL-DE-24-07-29-18-01-40-001-BWL2'

In [11]:
erp_df['父SKU'] = erp_df['父SKU'].fillna(erp_df['SKU']).copy()
erp_df['parent_child'] = np.where(erp_df['父SKU'] == erp_df['SKU'], 'Parent', 'Child')

inuse_skus = open('./util/skus/inuse_skus.txt', mode='r', encoding='utf-8').readlines()
inuse_skus = [i.strip() for i in inuse_skus]
archived_skus = open('./util/skus/archived_skus.txt', mode='r', encoding='utf-8').readlines()
archived_skus = [i.strip() for i in archived_skus]
deleted_skus = open('./util/skus/deleted_skus.txt', mode='r', encoding='utf-8').readlines()
deleted_skus = [i.strip() for i in deleted_skus]
recalled_skus = open('./util/skus/recalled_skus.txt', mode='r', encoding='utf-8').readlines()
recalled_skus = [i.strip() for i in recalled_skus]
other_skus = open('./util/skus/other_skus.txt', mode='r', encoding='utf-8').readlines()
other_skus = [i.strip() for i in other_skus]

erp_df = erp_df[~erp_df['父SKU'].isin(inuse_skus + archived_skus + deleted_skus + recalled_skus + other_skus)]

ams = []
amps = []

for i, (n, g) in enumerate(erp_df.groupby(by=['父SKU'])):
    for index, row in g.iterrows():
        am_sku = generate_amazon_sku(row['SKU'], 'FRL', 'DE', i + 1)
        am_psku = generate_amazon_sku(row['父SKU'], 'FRL', 'DE', i + 1)
        ams.append(am_sku)
        amps.append(am_psku)

erp_df['amazon_sku'] = ams.copy()
erp_df['amazon_psku'] = amps.copy()
erp_df[['SKU', '父SKU', 'amazon_sku', 'amazon_psku']]

C:\Users\000\AppData\Local\Temp\ipykernel_12152\3870049452.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  erp_df['amazon_sku'] = ams.copy()
C:\Users\000\AppData\Local\Temp\ipykernel_12152\3870049452.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  erp_df['amazon_psku'] = amps.copy()


,SKU,父SKU,amazon_sku,amazon_psku
38,AWB231219002,AWB231219002,FRL-DE-24-07-29-18-01-40-001,FRL-DE-24-07-29-18-01-40-001
39,AWB231219002GNL,AWB231219002,FRL-DE-24-07-29-18-01-40-001-GNL,FRL-DE-24-07-29-18-01-40-001
40,AWB231219002GNL2,AWB231219002,FRL-DE-24-07-29-18-01-40-001-GNL2,FRL-DE-24-07-29-18-01-40-001
41,AWB231219002GNL3,AWB231219002,FRL-DE-24-07-29-18-01-40-001-GNL3,FRL-DE-24-07-29-18-01-40-001
42,AWB231219002GNM,AWB231219002,FRL-DE-24-07-29-18-01-40-001-GNM,FRL-DE-24-07-29-18-01-40-001
...,...,...,...,...
683,ZZS210511744GYL5,ZZS210511744,FRL-DE-24-07-29-18-01-40-038-GYL5,FRL-DE-24-07-29-18-01-40-038
684,ZZS210511744GYS,ZZS210511744,FRL-DE-24-07-29-18-01-40-038-GYS,FRL-DE-24-07-29-18-01-40-038
685,ZZS210511744KHL4,ZZS210511744,FRL-DE-24-07-29-18-01-40-038-KHL4,FRL-DE-24-07-29-18-01-40-038
686,ZZS210511744KHL5,ZZS210511744,FRL-DE-24-07-29-18-01-40-038-KHL5,FRL-DE-24-07-29-18-01-40-038


In [12]:
fp_title = f'{g}/{g}_title.xlsx'
title_df: DataFrame = pd.read_excel(fp_title, sheet_name='title')

from util.widgets import Status

title_unused_df = title_df[title_df['status'] == Status.UNUSED.value]

# 创建一个字典来跟踪每个 parent_sku 分配的 title
title_allocation = {}

# 遍历 title_df，按顺序为每个 parent_sku 分配 title
for i, row in title_unused_df.iterrows():
    title = row['title']
    # 如果所有的 parent_sku 都已分配 title，则跳出循环
    if set(title_allocation.keys()) == set(erp_df['父SKU'].unique()):
        break

    # 找到第一个尚未分配 title 的 parent_sku
    for parent_sku in erp_df['父SKU'].unique():
        if parent_sku not in title_allocation:
            title_allocation[parent_sku] = title
            break

# 更新 title_df 中被分配的 title 的状态
title_unused_df.loc[title_unused_df['title'].isin(title_allocation.values()), 'status'] = Status.IN_USE.value

# 将分配的 title 添加到 erp_df 中
erp_df['title'] = erp_df['父SKU'].map(title_allocation)
index = title_unused_df[title_unused_df['status'] == Status.IN_USE.value].index
title_df.iloc[index, title_df.columns.get_loc('status')] = Status.IN_USE.value
title_df.iloc[index, title_df.columns.get_loc('ymd_HMS')] = ymd_HMS

ValueError: Protocol not known:                   SKU          父SKU  产品缩略图  \
678      ZZS210511744  ZZS210511744    NaN   
679  ZZS210511744AGL4  ZZS210511744    NaN   
680  ZZS210511744AGL5  ZZS210511744    NaN   
681   ZZS210511744BKS  ZZS210511744    NaN   
682  ZZS210511744GYL4  ZZS210511744    NaN   
683  ZZS210511744GYL5  ZZS210511744    NaN   
684   ZZS210511744GYS  ZZS210511744    NaN   
685  ZZS210511744KHL4  ZZS210511744    NaN   
686  ZZS210511744KHL5  ZZS210511744    NaN   
687   ZZS210511744KHS  ZZS210511744    NaN   

                                                  产品类型  \
678  普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...   
679  普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...   
680  普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...   
681  普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...   
682  普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...   
683  普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...   
684  普通产品 -> Clothing Shoes & Accessories -> 男装 -> ... 

In [ ]:
fp_st250 = f'{g}/{g}_st250.xlsx'
st250_df: DataFrame = pd.read_excel(fp_st250, sheet_name='st250')

from util.widgets import Status

st250_unused_df = st250_df[st250_df['status'] == Status.UNUSED.value]

# 创建一个字典来跟踪每个 parent_sku 分配的 st250
st250_allocation = {}

# 遍历 st250_df，按顺序为每个 parent_sku 分配 st250
for i, row in st250_unused_df.iterrows():
    value = row['value']
    # 如果所有的 parent_sku 都已分配 title，则跳出循环
    if set(st250_allocation.keys()) == set(erp_df['父SKU'].unique()):
        break

    # 找到第一个尚未分配 title 的 parent_sku
    for parent_sku in erp_df['父SKU'].unique():
        if parent_sku not in st250_allocation:
            st250_allocation[parent_sku] = value
            break

# 更新 st250_df 中被分配的 title 的状态
st250_unused_df.loc[st250_unused_df['value'].isin(st250_allocation.values()), 'status'] = Status.IN_USE.value

# 将分配的 title 添加到 erp_df 中
erp_df['st250'] = erp_df['父SKU'].map(st250_allocation)
index = st250_unused_df[st250_unused_df['status'] == Status.IN_USE.value].index
st250_df.iloc[index, st250_df.columns.get_loc('status')] = Status.IN_USE.value
st250_df.iloc[index, st250_df.columns.get_loc('ymd_HMS')] = ymd_HMS

In [ ]:
"""
面料/材质：
特征/设计：
搭配：
适用场合：
客户服务：

【轻质材料】优质涤纶。轻盈、柔软亲肤，舒适、透气、轻便，即使在夏季也能舒适慢跑。
【实用设计】运动修身健身运动裤，具有良好的弹性、透气性、重量轻、无摩擦和刺激，腿部贴合，展现肌肉张力并提供最大的运动自由度。
【易于搭配】我们的男士工装裤可以与夹克、毛衣等搭配，冬天保暖又时尚。这款裤子也可以搭配T恤和衬衫。一年四季必备的裤子。
【更多用途】适合休闲、运动、旅行、户外、家居、其他非正式活动或商务场合等，也可以在日常生活和工作中穿着。这款裤子也是送给朋友、儿子、丈夫或自己的完美礼物。
【注意】订购前，请参考产品说明中我们的详细尺码表（不是亚马逊尺码表）。由于手工测量，请允许1-2厘米的误差。我们尽力向您展示正确的颜色，但请接受由于显示器和灯光不同而出现的一些轻微色差。

描述：需要转成html格式
【特征】
材质：聚酯纤维
包装内容：1 条工装裤
风格: 休闲
性别：男
合身：宽松
长度：长裤
季节: 春季，夏季，秋季，冬季
【尺寸信息】
erp_df['尺码表']
【客户服务】
我们致力于为客户提供舒适的购物体验。如果您有任何疑问，请写信给我们，您将得到完美的解决方案。
"""

bullet_point1 = '【Leichtes Material】Hochwertiges Polyester. Leicht, weich und hautfreundlich, komfortabel, atmungsaktiv und leicht, ideal auch für das Joggen im Sommer.'
bullet_point2 = '【Praktisches Design】Sportliche, figurbetonte Trainingshose mit guter Elastizität, Atmungsaktivität, leichtem Gewicht, ohne Reibung und Irritationen. Sie passt sich eng an die Beine an, zeigt die Muskelspannung und bietet maximale Bewegungsfreiheit.'
bullet_point3 = '【Einfach zu kombinieren】Unsere Herren Cargohose kann mit Jacken, Pullovern usw. kombiniert werden, um im Winter warm und modisch zu bleiben. Diese Hose kann auch mit T-Shirts und Hemden getragen werden. Eine Hose, die das ganze Jahr über ein Muss ist.'
bullet_point4 = '【Vielfältige Verwendungsmöglichkeiten】Geeignet für Freizeit, Sport, Reisen, Outdoor, Zuhause, andere informelle Anlässe oder Geschäftstreffen. Diese Hose ist auch ein perfektes Geschenk für Freunde, Söhne, Ehemänner oder sich selbst.'
bullet_point5 = '【Hinweis】Bitte beachten Sie vor der Bestellung unsere detaillierte Größentabelle in der Produktbeschreibung (nicht die Amazon-Größentabelle). Aufgrund manueller Messungen erlauben Sie bitte einen Fehler von 1-2 cm. Wir bemühen uns, Ihnen die richtigen Farben zu zeigen, aber akzeptieren Sie bitte leichte Farbabweichungen aufgrund unterschiedlicher Monitore und Beleuchtung.'


def generate_pd(t: str):
    """
    Größe: S Taille: 78-84cm/30.71-33.07'' Hüfte: 100cm/39.37'' Länge: 66cm/25.98''
    :param t:
    :return:
    """
    de_map = {
        'Size': 'Größe',
        'Waist': 'Taille',
        'Hip': 'Hüfte',
        'Length': 'Länge',
    }
    for k, v in de_map.items():
        t = t.replace(k, v)
    cs = t.split('\n')
    pd = f"""<p><b>Merkmale</b><br></p>
<p>Material: Polyester</p>
<p>Verpackungsinhalt: 1 Cargo-Hose</p>
<p>Stil: Freizeit</p>
<p>Geschlecht: Herren</p>
<p>Passform: Locker</p>
<p>Länge: Lange Hose</p>
<p>Saison: Frühling, Sommer, Herbst, Winter</p>
<p><b>Größeninformationen</b></p>
{''.join([f'<p>{c.strip()}</p>' for c in cs])}
<p><b>Kundendienst</b></p>
<p>Wir sind bestrebt, unseren Kunden ein komfortables Einkaufserlebnis zu bieten. Wenn Sie Fragen haben, schreiben Sie uns bitte, und Sie erhalten eine perfekte Lösung.</p>"""
    return pd.replace('\n', '')


erp_df['product_description'] = erp_df['尺码表'].apply(lambda i: generate_pd(i))
erp_df